# Feinabstimmung von Open AI Modellen

Dieses Notebook basiert auf den aktuellen Richtlinien in der [Feinabstimmung](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) Dokumentation von Open AI.

Feinabstimmung verbessert die Leistung von Basismodellen für Ihre Anwendung, indem es mit zusätzlichen Daten und Kontext, die für diesen speziellen Anwendungsfall oder dieses Szenario relevant sind, erneut trainiert wird. Beachten Sie, dass Prompt-Engineering-Techniken wie _few shot learning_ und _retrieval augmented generation_ es Ihnen ermöglichen, den Standard-Prompt mit relevanten Daten zu erweitern, um die Qualität zu verbessern. Diese Ansätze sind jedoch durch die maximale Token-Fenstergröße des Ziel-Basismodells begrenzt.

Mit der Feinabstimmung trainieren wir das Modell effektiv selbst mit den erforderlichen Daten neu (was uns erlaubt, viel mehr Beispiele zu verwenden, als in das maximale Token-Fenster passen) – und setzen eine _benutzerdefinierte_ Version des Modells ein, die beim Inferenzzeitpunkt keine Beispiele mehr benötigt. Dies verbessert nicht nur die Effektivität unseres Prompt-Designs (wir haben mehr Flexibilität bei der Nutzung des Token-Fensters für andere Dinge), sondern verbessert potenziell auch unsere Kosten (indem die Anzahl der Tokens, die wir beim Inferenzzeitpunkt an das Modell senden müssen, reduziert wird).

Die Feinabstimmung umfasst 4 Schritte:
1. Bereiten Sie die Trainingsdaten vor und laden Sie sie hoch.
1. Führen Sie den Trainingsjob aus, um ein feinabgestimmtes Modell zu erhalten.
1. Bewerten Sie das feinabgestimmte Modell und iterieren Sie zur Qualitätsverbesserung.
1. Setzen Sie das feinabgestimmte Modell für die Inferenz ein, wenn Sie zufrieden sind.

Beachten Sie, dass nicht alle Basismodelle Feinabstimmung unterstützen – [prüfen Sie die OpenAI-Dokumentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) für die neuesten Informationen. Sie können auch ein zuvor feinabgestimmtes Modell erneut feinabstimmen. In diesem Tutorial verwenden wir `gpt-35-turbo` als unser Ziel-Basismodell für die Feinabstimmung.

---


### Schritt 1.1: Bereiten Sie Ihren Datensatz vor

Lassen Sie uns einen Chatbot erstellen, der Ihnen hilft, das Periodensystem der Elemente zu verstehen, indem er Fragen zu einem Element mit einem Limerick beantwortet. In _diesem_ einfachen Tutorial erstellen wir nur einen Datensatz, um das Modell mit einigen Beispielantworten zu trainieren, die das erwartete Format der Daten zeigen. In einem realen Anwendungsfall müssten Sie einen Datensatz mit viel mehr Beispielen erstellen. Möglicherweise können Sie auch einen offenen Datensatz (für Ihre Anwendungsdomäne) verwenden, falls ein solcher existiert, und ihn für die Feinabstimmung umformatieren.

Da wir uns auf `gpt-35-turbo` konzentrieren und eine Einzelantwort (Chat Completion) suchen, können wir Beispiele mit [diesem vorgeschlagenen Format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) erstellen, das den Anforderungen der OpenAI-Chat-Completion entspricht. Wenn Sie mehrstufige Konversationsinhalte erwarten, würden Sie das [Mehrstufige-Beispiel-Format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) verwenden, das einen `weight`-Parameter enthält, um anzugeben, welche Nachrichten im Feinabstimmungsprozess verwendet werden sollen (oder nicht).

Für unser Tutorial verwenden wir das einfachere Einzelantwort-Format. Die Daten liegen im [jsonl-Format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) mit einem Datensatz pro Zeile vor, wobei jeder als JSON-formatiertes Objekt dargestellt wird. Der Ausschnitt unten zeigt 2 Datensätze als Beispiel – siehe [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) für den vollständigen Beispielsatz (10 Beispiele), den wir für unser Feinabstimmungstutorial verwenden. **Hinweis:** Jeder Datensatz _muss_ in einer einzigen Zeile definiert sein (nicht über mehrere Zeilen verteilt, wie es in einer formatierten JSON-Datei üblich ist).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In einem realen Anwendungsfall benötigen Sie eine viel größere Beispielsammlung für gute Ergebnisse – der Kompromiss liegt zwischen der Qualität der Antworten und der Zeit/Kosten für die Feinabstimmung. Wir verwenden eine kleine Sammlung, damit wir die Feinabstimmung schnell abschließen können, um den Prozess zu veranschaulichen. Siehe [dieses OpenAI Cookbook-Beispiel](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) für ein komplexeres Feinabstimmungstutorial.


---

### Schritt 1.2 Laden Sie Ihren Datensatz hoch

Laden Sie die Daten mit der Files API [wie hier beschrieben](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) hoch. Beachten Sie, dass Sie, um diesen Code auszuführen, zuvor die folgenden Schritte durchgeführt haben müssen:
 - Das Python-Paket `openai` installiert haben (stellen Sie sicher, dass Sie eine Version >=0.28.0 für die neuesten Funktionen verwenden)
 - Die Umgebungsvariable `OPENAI_API_KEY` auf Ihren OpenAI-API-Schlüssel gesetzt haben
Um mehr zu erfahren, siehe die [Setup-Anleitung](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), die für den Kurs bereitgestellt wird.

Führen Sie nun den Code aus, um eine Datei für den Upload aus Ihrer lokalen JSONL-Datei zu erstellen.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Schritt 2.1: Erstellen Sie den Fine-Tuning-Job mit dem SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Schritt 2.2: Überprüfen Sie den Status des Jobs

Hier sind einige Dinge, die Sie mit der `client.fine_tuning.jobs` API tun können:
- `client.fine_tuning.jobs.list(limit=<n>)` - Listet die letzten n Fine-Tuning-Jobs auf
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Ruft Details zu einem bestimmten Fine-Tuning-Job ab
- `client.fine_tuning.jobs.cancel(<job_id>)` - Bricht einen Fine-Tuning-Job ab
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Listet bis zu n Ereignisse des Jobs auf
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Der erste Schritt des Prozesses ist die _Validierung der Trainingsdatei_, um sicherzustellen, dass die Daten im richtigen Format vorliegen.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Schritt 2.3: Ereignisse verfolgen, um den Fortschritt zu überwachen


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Schritt 2.4: Status im OpenAI-Dashboard anzeigen


Sie können den Status auch einsehen, indem Sie die OpenAI-Website besuchen und den Abschnitt _Fine-tuning_ der Plattform erkunden. Dort wird Ihnen der Status des aktuellen Jobs angezeigt und Sie können auch die Historie vorheriger Jobausführungen verfolgen. In diesem Screenshot sehen Sie, dass die vorherige Ausführung fehlgeschlagen ist und der zweite Lauf erfolgreich war. Zum Kontext: Dies geschah, als der erste Lauf eine JSON-Datei mit falsch formatierten Datensätzen verwendete – nachdem dies behoben wurde, wurde der zweite Lauf erfolgreich abgeschlossen und das Modell zur Nutzung freigegeben.

![Fine-tuning job status](../../../../../translated_images/de/fine-tuned-model-status.563271727bf7bfba.webp)


Sie können die Statusmeldungen und Metriken auch anzeigen, indem Sie im visuellen Dashboard weiter nach unten scrollen, wie gezeigt:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/de/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/de/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Schritt 3.1: ID abrufen & Feinabgestimmtes Modell im Code testen


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Schritt 3.2: Feinabgestimmtes Modell im Playground laden und testen

Sie können das feinabgestimmte Modell jetzt auf zwei Arten testen. Zuerst können Sie den Playground besuchen und im Dropdown-Menü „Models“ Ihr neu feinabgestimmtes Modell aus der Liste auswählen. Die andere Möglichkeit ist die Verwendung der Option „Playground“, die im Fine-tuning-Bereich angezeigt wird (siehe Screenshot oben). Diese startet die folgende _vergleichende_ Ansicht, die die Foundation- und die feinabgestimmte Modellversion nebeneinander für eine schnelle Bewertung anzeigt.

![Fine-tuning job status](../../../../../translated_images/de/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Füllen Sie einfach den im Training verwendeten Systemkontext aus und geben Sie Ihre Testfrage ein. Sie werden feststellen, dass auf beiden Seiten der identische Kontext und die Frage aktualisiert werden. Führen Sie den Vergleich aus und Sie sehen die Unterschiede in den Ausgaben zwischen den Modellen. _Beachten Sie, wie das feinabgestimmte Modell die Antwort im von Ihnen in den Beispielen vorgegebenen Format darstellt, während das Foundation-Modell einfach der Systemaufforderung folgt_.

![Fine-tuning job status](../../../../../translated_images/de/fine-tuned-playground-launch.5a26495c983c6350.webp)

Sie werden feststellen, dass der Vergleich auch die Token-Anzahl für jedes Modell sowie die für die Inferenz benötigte Zeit anzeigt. **Dieses spezielle Beispiel ist ein einfaches, das den Prozess zeigen soll, aber keine reale Datensatz- oder Szenarioabbildung darstellt**. Sie werden bemerken, dass beide Beispiele die gleiche Anzahl an Tokens zeigen (Systemkontext und Benutzeraufforderung sind identisch), wobei das feinabgestimmte Modell mehr Zeit für die Inferenz benötigt (benutzerdefiniertes Modell).

In realen Szenarien werden Sie kein Spielzeugbeispiel wie dieses verwenden, sondern gegen echte Daten feinabstimmen (z. B. Produktkatalog für den Kundenservice), bei denen die Qualität der Antwort viel deutlicher wird. In _diesem_ Kontext erfordert das Erreichen einer gleichwertigen Antwortqualität mit dem Foundation-Modell mehr individuelles Prompt-Engineering, was den Token-Verbrauch und potenziell die damit verbundene Verarbeitungszeit für die Inferenz erhöht. _Um dies auszuprobieren, sehen Sie sich die Fine-Tuning-Beispiele im OpenAI Cookbook an, um zu starten._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ursprungssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
